# EEG DataLoader

The dimensions of the training set are as follows: 4,500 samples, 64 channels, and a time length of 795. This corresponds to 5 categories in y_train.

The dimensions of the testing set are as follows: 750 samples, 64 channels, and a time length of 795. This corresponds to 5 categories in y_test.

You can download it from this Google Drive link: [https://drive.google.com/drive/folders/1ykR-mn4d4KfFeeNrfR6UdtebsNRY8PU2?usp=sharing].
Please download the data and place it in your data_path at "./data."

In [ ]:
import numpy as np
import torch
import os
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data_path =  '/content/drive/MyDrive/'

In [5]:
train_data = np.load(data_path + 'train_data.npy')
test_data = np.load(data_path + 'test_data.npy')
train_label = np.load(data_path + 'train_label.npy')
test_label = np.load(data_path + 'test_label.npy')

#To convert the data into PyTorch tensors
x_train_tensor = torch.Tensor(train_data)
y_train_tensor = torch.LongTensor(train_label)
x_test_tensor = torch.Tensor(test_data)
y_test_tensor = torch.LongTensor(test_label)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #Setting GPU on your computer

In [7]:
train_dataset = TensorDataset(x_train_tensor.to(device), y_train_tensor.to(device)) # input data to Tensor dataloader
train_loader = DataLoader(train_dataset, batch_size=64, drop_last=True, shuffle=True) #  Batch size refers to the number of data sample
test_dataset = TensorDataset(x_test_tensor.to(device), y_test_tensor.to(device))
test_loader = DataLoader(test_dataset, batch_size=64,  drop_last=True,shuffle=False)

# Build simple Deep learning model

In [8]:
class EEGAutoencoderClassifier(nn.Module):
    def __init__(self, num_classes):
        super(EEGAutoencoderClassifier, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(64 * 795, 256), # Input dimention is 64 channel * 795 time point, and use 256 units for first NN layer
            nn.ReLU(), # Use ReLu function for NN training
            nn.Linear(256, 128), # 256 NN units to 128 units
            nn.ReLU(),
            nn.Linear(128, 64),#  128 NN units to 64 units
            nn.ReLU()
        )
        self.classifier = nn.Sequential(
            nn.Linear(64, num_classes), # num_classes is 5 (hello,” “help me,” “stop,” “thank you,” and “yes”)
            nn.LogSoftmax(dim=1)  # Use LogSoftmax for multi-class classification
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.encoder(x)

        # import pdb;pdb.set_trace()
        x = self.classifier(x)
        return x

In [9]:
num_classes = 5 # setting final output class
model = EEGAutoencoderClassifier(num_classes).to(device)
criterion = nn.NLLLoss() # Use NLLLoss function to optimize
optimizer = optim.Adam(model.parameters(), lr=0.001) # Setting parameters learning rate = 0.001

In [11]:
num_epochs = 200 # setting training epochs (Number of training iterations)
for epoch in range(num_epochs):
    model.train()
    for data, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')

Epoch 1/200, Loss: 1.5460231304168701
Epoch 2/200, Loss: 1.4746407270431519
Epoch 3/200, Loss: 1.4618765115737915
Epoch 4/200, Loss: 1.405974268913269
Epoch 5/200, Loss: 1.2254456281661987
Epoch 6/200, Loss: 1.0531904697418213
Epoch 7/200, Loss: 1.1934298276901245
Epoch 8/200, Loss: 1.0638296604156494
Epoch 9/200, Loss: 0.7753558158874512
Epoch 10/200, Loss: 0.5620149374008179
Epoch 11/200, Loss: 0.5787872672080994
Epoch 12/200, Loss: 0.5097058415412903
Epoch 13/200, Loss: 0.31779080629348755
Epoch 14/200, Loss: 0.37457960844039917
Epoch 15/200, Loss: 0.36143070459365845
Epoch 16/200, Loss: 0.5005438923835754
Epoch 17/200, Loss: 0.13512133061885834
Epoch 18/200, Loss: 0.358467698097229
Epoch 19/200, Loss: 0.2624067962169647
Epoch 20/200, Loss: 0.34161293506622314
Epoch 21/200, Loss: 0.2708033323287964
Epoch 22/200, Loss: 0.09066638350486755
Epoch 23/200, Loss: 0.171726256608963
Epoch 24/200, Loss: 0.3261389434337616
Epoch 25/200, Loss: 0.12000444531440735
Epoch 26/200, Loss: 0.17198684

In [12]:
model.eval() # Evaluate your model
correct = 0
total = 0

with torch.no_grad():
    for data, labels in test_loader:
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Test Accuracy: {accuracy * 100:.2f}%')

Test Accuracy: 49.15%


In [20]:
import joblib
from torch import nn

In [21]:
joblib.dump(model,'/content/drive/MyDrive/Saved Models/BCI_200')

['/content/drive/MyDrive/Saved Models/BCI_200']